# PROCESS SUBMISSIONS CSV
The purpose of this script is to get metrics on all submissions and the subset of chosen submissions. This subset will be used for further analysis of submitted films. All data is extracted from local flat files.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter
import datetime as dt

In [2]:
subs_df = pd.read_csv("data/filmfreeway-submissions.csv")

In [3]:
subs_df.info()
print(f'{subs_df.shape[0]} rows & {subs_df.shape[1]} columns.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Project Title          103 non-null    object 
 1   Duration               103 non-null    object 
 2   Genres                 40 non-null     object 
 3   First Name             103 non-null    object 
 4   Last Name              103 non-null    object 
 5   Email                  103 non-null    object 
 6   City                   67 non-null     object 
 7   State                  65 non-null     object 
 8   Country                67 non-null     object 
 9   Submission Date        103 non-null    object 
 10  Submission Status      103 non-null    object 
 11  Submission Categories  103 non-null    object 
 12  Rating                 0 non-null      float64
 13  Submission ID          103 non-null    int64  
 14  Birthdate              83 non-null     object 
 15  Gender

In [4]:
subs_df.isnull().sum()

Project Title              0
Duration                   0
Genres                    63
First Name                 0
Last Name                  0
Email                      0
City                      36
State                     38
Country                   36
Submission Date            0
Submission Status          0
Submission Categories      0
Rating                   103
Submission ID              0
Birthdate                 20
Gender                    13
Judging Status             0
Submission Link            0
Directors                  2
Writers                   18
Producers                 20
dtype: int64

In [ ]:
print(set(x for x in subs_df['Submission Status']))

## CLEAN DATA
Note: Should "Incomlete" entries be dropped?

In [5]:
# Drop withdrawn entries
subs_df = subs_df[subs_df['Submission Status'] != 'Withdrawn']

In [6]:
# Replace column to shorten: Feel Good Shorts
subs_df['Submission Categories'] = subs_df['Submission Categories'].apply( \
        lambda x: x.replace('Short films that make you feel good.', 'Feel Good Shorts'))

In [7]:
# 'Youth Short Films, Short Documentary' force to 'Short Documentary'
subs_df['Submission Categories'] = subs_df['Submission Categories'].apply( \
        lambda x: x.replace('Youth Short Films, Short Documentary', 'Short Documentary'))

In [8]:
# drop rating, genres cols due to all missing data
subs_df.drop(columns=['Rating', 'Genres'], inplace=True)

In [9]:
# fill na with empty str to allow apply method
subs_df['State'].fillna('', inplace=True)

In [10]:
# replace to unify state names
subs_df['State'] = subs_df['State'].apply(
    lambda x: x.replace('maryland', 'MD')
               .replace('Maryland', 'MD')
               .replace('Virginia', 'VA')
               .replace('Georgia', 'GA')
               .replace('Pennsylvania', 'PA')
               .replace('Florida', 'FL')
               .replace('New York', 'NY')
               .replace('ny', 'NY')
)

## Subset Selected Entries

In [11]:
selected_df = subs_df[subs_df['Judging Status']=='Selected']
print(f'Acceptance Rate: {len(selected_df) / len(subs_df)}, ({len(selected_df)} / {len(subs_df)})')

Acceptance Rate: 0.72, (72 / 100)


## EDA

In [ ]:
selected_df.columns

In [ ]:
selected_gender_count = Counter(selected_df.Gender)
print(f'Gender Count: {selected_gender_count}', '\n')

In [ ]:
state_count = Counter(selected_df.State)
us_state_count = [state for state in state_count.items() if len(state[0]) == 2]
international_count = [state for state in state_count.items() if len(state[0]) != 2 and state[0] != '']
unknown_loc_count = [state for state in state_count.items() if state[0] == '']
print('US COUNT:')
print(us_state_count)
print('\nINTERNATIONAL COUNT')
print(international_count)
print('\nUNKNOWN COUNT')
print(unknown_loc_count)

In [ ]:
def make_bar(y_pos, n, labels, xlabel, filename):
        plt.rcdefaults()
        fig, ax = plt.subplots()
        _ = ax.barh(y_pos, n, align='center')
        _ = ax.set_yticks(y_pos)
        _ = ax.set_yticklabels(labels)
        _ = ax.invert_yaxis()
        _ = ax.set_xlabel(str(xlabel))
        _ = ax.set_title(str(filename))
        _ = plt.tight_layout()
        # plt.savefig(DATA_FOLDER / f'{filename}.png')
        plt.show()

In [ ]:
def make_pie(ratios, labels, filename):
        fig1, ax = plt.subplots()
        _ = ax.pie(ratios, labels=labels, autopct='%1.1f%%', shadow=False, normalize=True)
        _ = ax.set_title(str(filename))
        _ = ax.axis('equal') # make sure pie is circle
        # plt.savefig(DATA_FOLDER / '{filename}.png', bbox_inches='tight')
        plt.show()

### PLOT ALL SUBMISSIONS PER CATEGORY

In [ ]:
subs_per_cat = dict(Counter(selected_df['Submission Categories']))
sorted_cat = sorted(subs_per_cat.items(), key=lambda x: x[1])
total_selected = len(selected_df['Project Title'])
labels = [k for k, v in sorted_cat]
number_of_entries = [v for k, v in sorted_cat]
ratios = [(v / total_selected) for k, v in sorted_cat]

make_bar(
        y_pos=np.arange(len(labels)), n=number_of_entries,
        labels=labels, xlabel='Number of Sumbissions', filename='EntriesPerCat_BAR'
         )

make_pie(
        ratios=ratios, labels=labels, filename='EntriesPerCat_PIE'
        )

### DURATION METRICS

In [12]:
# Convert Duration column to timedelta datatype
selected_df.Duration = pd.to_timedelta(selected_df.Duration)

In [40]:
selected_duration = selected_df.loc[:, ('Submission Categories', 'Duration')]
selected_duration

,Submission Categories,Duration
1,Dramatic Shorts,0 days 00:17:45
2,Dramatic Shorts,0 days 00:00:00
3,Feature Films,0 days 01:22:00
6,Youth Short Films,0 days 00:09:59
7,Short Documentary,0 days 00:12:03
...,...,...
97,Horror/Sci-fi,0 days 00:11:09
98,Feature Films,0 days 00:24:42
99,Dramatic Shorts,0 days 00:13:00
101,Funny Shorts,0 days 00:15:00


In [31]:
# Get Duration sum per category
total_duration = selected_df.Duration.sum()
print(f'Total Duration of Films Selected: {total_duration}')

Total Duration of Films Selected: 0 days 20:38:31


In [41]:
duration_per_cat = selected_duration.groupby('Submission Categories').sum()
duration_per_cat['count'] = selected_duration.groupby('Submission Categories').count()
duration_per_cat

,Duration,count
Submission Categories,,
Animation,0 days 00:50:32,11
Dramatic Shorts,0 days 02:02:50,10
Environmental & Aquatic Films,0 days 00:30:32,4
Feature Films,0 days 09:22:04,9
Feel Good Shorts,0 days 00:55:04,5
Funny Shorts,0 days 01:11:28,6
Horror/Sci-fi,0 days 02:01:31,11
Short Documentary,0 days 03:09:52,11
Social Commentary Shorts,0 days 00:09:32,1


In [ ]:
mean_duration = {}
for pair, v in selected_duration.items():
    cat, stat = pair
    if stat == 'mean':
        hours, remainder = divmod(v, 3600)
        minutes, seconds = divmod(remainder, 60)
        mean_duration[cat] = '{:02}:{:02}:{:02}'.format(int(hours), int(minutes), int(seconds))

print(mean_duration)